In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os.path
import numpy as np
import pandas as pd
import dateutil
import datetime
import re
import matplotlib.pyplot as plt

import load_data

# pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)
#test

In [ ]:
raw_data = load_data.load_das_data('./data/', 'Capacity test Raw data- 2017-03-25_7 PM.csv')

In [ ]:
#raw_data.columns.tolist()

In [ ]:
# move to copy of this for canal data
# canal_das = load_das_data('./data/', 'Data_Extract_20170807-20170807_cap_test_daily_id38778c9c-115.csv')

In [ ]:
col_types = raw_data.apply(load_data.series_type, args=(load_data.type_defs,)).tolist()

In [ ]:
sub_types = raw_data.apply(load_data.series_type, args=(load_data.sub_type_defs,), bounds_check=False).tolist()

In [ ]:
irr_types = raw_data.apply(load_data.series_type, args=(load_data.irr_sensors_defs,), bounds_check=False).tolist()

In [ ]:
col_indices = ['-'.join([typ, sub_typ, irr_typ]) for typ, sub_typ, irr_typ in zip(col_types, sub_types, irr_types)]

In [ ]:
raw_data_clean_headers = raw_data.copy()
raw_data_clean_headers.columns = col_indices

In [ ]:
eq_cnts = load_data.equip_counts(raw_data_clean_headers)

In [ ]:
len(raw_data_clean_headers.columns.tolist())

In [ ]:
len(eq_cnts)

In [ ]:
n_col_names = list(zip(col_indices, eq_cnts))
raw_data_clean_headers.columns = pd.MultiIndex.from_tuples(n_col_names, names=['label','eq_cnt'])

In [ ]:
raw_data_clean_headers.head()

In [ ]:
raw_data_clean_headers.groupby(level='label', axis=1).mean()

In [ ]:
# blythe_das.iloc[:,3].plot().legend(loc='center left', bbox_to_anchor=(1,0.5))